In [1]:
#!pip install --upgrade pip
# !pip install --upgrade "pip<24.1"


In [2]:
#!pip install fairseq #for finetuning\
!pip install --upgrade datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import torch

In [4]:
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
import json


processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

/home/kris/Documents/fyp/fyp_env/lib/python3.11/site-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [5]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torchaudio
import torch

# Load the Wav2Vec2 model (pre-trained but not fine-tuned)
model_name = "facebook/wav2vec2-base"
model = Wav2Vec2ForCTC.from_pretrained(model_name)

/home/kris/Documents/fyp/fyp_env/lib/python3.11/site-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:

# Load an audio file
waveform, sample_rate = torchaudio.load("../creolese-audio-dataset/Audio Files/3.wav")

# Convert to 16kHz (Wav2Vec2 requires this)
if sample_rate != 16000:
    waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

# Process the audio
input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", padding=True, sampling_rate=16000).input_values

# Get predictions
with torch.no_grad():
    logits = model(input_values).logits

# Convert predictions to text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print(processor.tokenizer.get_vocab())


print("Transcription:", transcription[0])  # Output the text


{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, 'E': 5, 'T': 6, 'A': 7, 'O': 8, 'N': 9, 'I': 10, 'H': 11, 'S': 12, 'R': 13, 'D': 14, 'L': 15, 'U': 16, 'M': 17, 'W': 18, 'C': 19, 'F': 20, 'G': 21, 'Y': 22, 'P': 23, 'B': 24, 'V': 25, 'K': 26, "'": 27, 'X': 28, 'J': 29, 'Q': 30, 'Z': 31}
Transcription: KIKIYGIYBIYIKIHBIKIKIKIKIKIPK IKIKIK KIKIK KIK KIKBK YBPIBHIUJBSBIUBIKUIJBIKYGKIJBKBJ IYR I YU IYR Y KIKYIBYIYKIPUKBIKGIKBE KI I IK EIPUB</s>ILIJBK IPIJNBHIKIKYKBKIPGIYGIKIYBIKIKIKIKIKIPIPKIKIPIKYKIYGIYIPJIJKIGIKIJKGIGBIKLGIKNGIPIPIJYIGBGKPIYGPIPYGPJPYIYG KGIPI YBIKIKPIKPIPKIKIKIKIJPKPIKIPKIKPKIPIPIPKIKIKPEIPIPIPKIK IJYUKIYK KUYKIJIPUKOIKPIJYPIPKY<s>IJBIEVIY PIBEPIPKIKIKIKIKJPKIKIPKIPIPIKIKPGIJIPKPIKIKIKIKIKIKIK</s>K<s>YG PIY KIPIYKIY IJKY KIYIYJI JIEIKIKIYKJIKIGIYBJIUI KIKIKPKPKPEF IKPKIPKIEIKIK VPIYJXUK<s>KHJY UI YPIYKIPGEIKIJIBJA KIKYIYG IPYKIKIKIJBJIKENCYKIYGIGYPJ JIPYGIPIYKIKIKIKPKPIKGIKIKIKPKHBEIKIGKIJBIPIKBJYGPJIPIPIJIJIKYKJIJKYJKIK KIGIPBUKIKIY I IEKIYK K<unk>KIK<unk>K<unk>K I

# Load the dataset

In [7]:
audio_path = "../creolese-audio-dataset/finetune_eligible"
transcription_path = "../creolese-audio-dataset/finetune_eligible/transcripts.json"

# Start compiling the references and hypotheses

In [8]:
import os
import json
import torchaudio
import torch
from jiwer import wer, cer, mer

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [9]:

with open(transcription_path, "r") as f:
    dataset = json.load(f)

predictions = []
ground_truths = []
device = "cpu"
for item in dataset:  # your dataset is a list of {audio, text, language}
        # Load audio file
    waveform, sample_rate = torchaudio.load(f'./{audio_path}/{item["audio"]}')

    # Convert to 16kHz (Wav2Vec2 requires this)
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)
    try:
        
        # Process the audio
        input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", padding=True, sampling_rate=16000).input_values
        
        # Get predictions
        with torch.no_grad():
            logits = model(input_values).logits
    
        predicted_ids = torch.argmax(logits, dim=-1)
        predicted_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        print(f'Transcribed: {item["audio"]}')

    except Exception as e:
        print(f"Exception: {e}")
    
    
    predictions.append(predicted_text)
    ground_truths.append(item["text"])
    
    # Manually clear memory each iteration
    del input_values, predicted_ids
    import gc; gc.collect()


Transcribed: 1a.wav
Transcribed: 1b.wav
Transcribed: 1c.wav
Transcribed: 1d.wav
Transcribed: 1e.wav
Transcribed: 3a.wav
Transcribed: 3b.wav
Transcribed: 2.wav
Transcribed: 4a.wav
Transcribed: 4b.wav
Transcribed: 4c.wav
Transcribed: 4d.wav
Transcribed: 4e.wav
Transcribed: 5a.wav
Transcribed: 5b.wav
Transcribed: 5c.wav
Transcribed: 5d.wav
Transcribed: 5e.wav
Transcribed: 5f.wav
Transcribed: 5g.wav
Transcribed: 5h.wav
Transcribed: 5i.wav
Transcribed: 5j.wav
Transcribed: 5k.wav
Transcribed: 5l.wav
Transcribed: 5m.wav
Transcribed: 6a.wav
Transcribed: 6b.wav
Transcribed: 6c.wav
Transcribed: 6d.wav
Transcribed: 6e.wav
Transcribed: 6f.wav
Transcribed: 6g.wav
Transcribed: 6h.wav
Transcribed: 6i.wav
Transcribed: 6j.wav
Transcribed: 6k.wav
Transcribed: 6l.wav
Transcribed: 6m.wav
Transcribed: 6n.wav
Transcribed: 6o.wav
Transcribed: 6p.wav
Transcribed: 7.wav
Transcribed: 8a.wav
Transcribed: 8b.wav
Transcribed: 8c.wav
Transcribed: 8d.wav
Transcribed: 8e.wav
Transcribed: 8f.wav
Transcribed: 8g.wav
Tr

In [10]:

output_data = [{"ref": ref, "hyp": hyp} for ref, hyp in zip(ground_truths, predictions)]

with open("../pretrained_predictions/wav2vec2_pretrain_predictions.json", "w") as out_file:
    json.dump(output_data, out_file, indent=4)

In [11]:
import json
import jiwer

# === STEP 1: LOAD JSON ===
with open("wav2vec2_pretrain_predictions.json", "r") as f:
    data = json.load(f)

ground_truth = [entry['ref'] for entry in data]
predictions = [entry['hyp'] for entry in data]     

In [12]:

def custom_transform(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation (basic approach)
    for char in ",.:;!?\"'()[]{}":
        text = text.replace(char, "")
    # Remove extra spaces
    text = " ".join(text.split())
    return text

# Apply transformation to each text individually
ground_truth_transformed = [custom_transform(text) for text in ground_truth]
predictions_transformed = [custom_transform(text) for text in predictions]

# === STEP 4: COMPUTE METRICS ===
wer = jiwer.wer(ground_truth_transformed, predictions_transformed)
cer = jiwer.cer(ground_truth_transformed, predictions_transformed)
mer = jiwer.mer(ground_truth_transformed, predictions_transformed)

# === STEP 5: PRINT ===
print(f"WER: {wer:.4f}")
print(f"CER: {cer:.4f}")
print(f"MER: {mer:.4f}")



WER: 0.9329
CER: 0.4517
MER: 0.9016
